In [ ]:
P = {}
P['EPOCHS'] = 30
P['BACKBONE'] = 'efficientnetb4'
P['NFOLDS'] = 4
P['SEED'] =  0
P['VERBOSE'] = 0
P['DISPLAY_PLOT'] = True
P['BATCH_CODE'] = 8

P['TILING'] = [1024, 512]
P['DIM'] = P['TILING'][1]
P['DIM_FROM'] = P['TILING'][0]

P['LR'] = 5e-4
P['OVERLAP'] = True
P['STEPS_COE'] = 3

import yaml
with open(r'params.yaml', 'w') as file:
    yaml.dump(P, file)

In [ ]:
!pip install segmentation_models -q
%matplotlib inline

import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

import glob
import segmentation_models as sm
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

from kaggle_datasets import KaggleDatasets
print("Tensorflow version" + tf.__version__)

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: 
    strategy = tf.distribute.get_strategy() 

BATCH_SIZE = P['BATCH_COE'] * strategy.num_replicas_in_sync

print("Number of accelerators: ", strategy.num_replicas_in_sync)
print("BATCH_SIZE: ", str(BATCH_SIZE))

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path(f'hubmap-tfrecords-1024-{P["DIM"]}')
ALL_TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*tfrec')
ALL_TRAINING_FILENAMES

In [ ]:
if P['OVERLAPP']:
    ALL_TRAINING_FILENAMES2 = tf.io.gfile.glob(GCS_PATH + '/train2/*tfrec')

In [ ]:
import re
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)
print('NUM_TRAINING_IMAGES: ')
if P['OVERLAPP']:
    print(count_data_items(ALL_TRAINING_FILENAMES2)+count_data_items(ALL_TRAINING_FILENAMES))
else:
    print(count_data_items(ALL_TRAINING_FILENAMES))        

In [ ]:
DIM = P['DIM']
def _parse_image_function(example_proto, augment = True):
    image_feature_description = {
        'image' : tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string)
    }
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.reshape()